<p align="center">
    <picture>
    <img alt="nerfstudio" src="https://dvic.devinci.fr/logos/dvic_logo.png" width="400">
    </picture>
</p>


# Tutorial: Create Photorrealistic 3D animations using NeRFs

This colab shows how to composite a Blender 3D animation with a NeRF video using Nerfstudio and Blender.

![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

Credit to [Nerfstudio]() for the base [Google Colab demo]().

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [ ]:
#@markdown <h1>Install Conda (requires runtime restart)</h1>

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:15
🔁 Restarting kernel...


In [ ]:
!pip install pandas \
  numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@markdown <h1>Install Nerfstudio and Dependencies (~15 min)</h1>

%cd /content/
!pip install -q --upgrade pip
!pip install -q torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=12RL_NVgE9WGvr_fEsXEiuaJ1QESvQCPl&confirm=t" 
!pip install -q tinycudann-1.7-cp38-cp38-linux_x86_64.whl

# Installing COLMAP
%cd /content/
!conda install -q -c conda-forge colmap

# Install nerfstudio
%cd /content/
# !pip install nerfstudio
!pip install -q git+https://github.com/nerfstudio-project/nerfstudio.git
!conda remove -q --force qt-main

/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 922.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 72.1 MB/s eta 0:00:00
/content
Downloading...
From: https://drive.google.com/u/1/uc?id=12RL_NVgE9WGvr_fEsXEiuaJ1QESvQCPl&confirm=t
To: /content/tinycudann-1.7-cp38-cp38-linux_x86_64.whl
100% 31.9M/31.9M [00:00<00:00, 209MB/s]
/content
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - colmap


The following packag

In [ ]:
#@markdown <h1> Downloading and Processing Data</h1>
#@markdown <h3>Pick the preset scene or upload your own images/video</h3>
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDDBC poster' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = ' '.join(scene.split(' ')[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == 'upload your own Record3D data':
    display(HTML('<h3>Zip your Record3D folder, and upload.</h3>'))
    display(HTML('<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'))
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob('/content/data/nerfstudio/custom_data/*')[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ['upload your images', 'upload your own video']:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on hires inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content
Downloading...
From: https://drive.google.com/uc?id=1dmjWGXlJnUxwosN6MVooCDQe970PkD-1
To: /content/data/nerfstudio/poster.zip
100% 750M/750M [00:04<00:00, 184MB/s]
Data Processing Succeeded!


In [ ]:
#@markdown <h1>Set up and Start Viewer</h1>

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")

from IPython import display
display.IFrame(src=url, height=800, width="100%")

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.916s
https://viewer.nerf.studio/?websocket_url=wss://icy-hands-wonder-34-69-7-233.loca.lt
You may need to click Refresh Page after you start training!


In [ ]:
#@markdown <h1>Start Training</h1>

%cd /content
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
  !ns-train nerfacto --experiment_name $scene --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
else:
    from IPython.core.display import display, HTML
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))

/content


NameError: ignored

In [ ]:
#@markdown <h1>Save trained model to Google Drive</h1>

# Use Google Drive to save our model and outputs
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

backup_filename = f'outputs_{scene}.zip'

# Compress nerfstudio outputs in a zipfile
%cd /content
!zip -r $backup_filename outputs/

# Copy a backup on your Google Drive
!mkdir -p '/content/gdrive/My Drive/nerfstudio'
!cp $backup_filename '/content/gdrive/My Drive/nerfstudio/'


In [ ]:
#@markdown <h1>*If you already have a pre-trained model*, load it from Google Drive</h1>

# Use Google Drive to load our pre-trained model and outputs
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

backup_filename = f'outputs_{scene}.zip'

# Load the saved nerfstudio outputs run:
%cd /content
!unzip 'gdrive/My Drive/nerfstudio/'$backup_filename

Mounted at /content/gdrive
/content
Archive:  gdrive/My Drive/nerfstudio/outputs_poster.zip
   creating: outputs/
   creating: outputs/poster/
   creating: outputs/poster/nerfacto/
   creating: outputs/poster/nerfacto/2023-03-02_162100/
   creating: outputs/poster/nerfacto/2023-03-02_162100/nerfstudio_models/
  inflating: outputs/poster/nerfacto/2023-03-02_162100/nerfstudio_models/step-000029999.ckpt  
  inflating: outputs/poster/nerfacto/2023-03-02_162100/viewer_log_filename.txt  
  inflating: outputs/poster/nerfacto/2023-03-02_162100/config.yml  
  inflating: outputs/poster/nerfacto/2023-03-02_162100/dataparser_transforms.json  


In [ ]:
#@title # Render Pointcloud
#@markdown <h3>Export the pointcloud generated from the NeRF model.</h3>
#@markdown <h5>The rendered pointcloud should be at exports/point_cloud.ply</h5>

%cd /content/
base_dir = f"outputs/{scene}/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]
config_filename = training_run_dir + "/config.yml"

!ns-export pointcloud --load-config $config_filename --output-dir exports/


✅ Cleaning Point Cloud
✅ Generated PointCloud with 1005393 points.
Saving Point Cloud...

✅ Saving Point Cloud


# Create your Bleder animation using the NeRF Pointcloud as a reference.

Return to the tutorial website for more detailed instructions.

In [ ]:
#@markdown <h1>Visualize pretrained model</h1>

base_dir = f"outputs/{scene}/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]
config_filename = training_run_dir + "/config.yml"
load_dir = training_run_dir + "/nerfstudio_models"

%cd /content
if os.path.exists(config_filename):
    !ns-train nerfacto --load-dir $load_dir --load-config $config_filename --viewer.start-train False
else:
    from IPython.core.display import display, HTML
    display(HTML('<h3 style="color:red">Error: Model training did not complete</h3>'))
    display(HTML('<h3>Please re-run `Start Training`, or check the model config path.</h3>'))

In [ ]:
#@title # Render Background Video { vertical-output: true }
#@markdown <h3>Export the camera path from Blender using the nerfstudio plugin, then run this cell.</h3>
#@markdown <h5>The rendered video should be at renders/output.mp4!</h5>

from IPython.core.display import display, HTML
display(HTML('<h3>Upload the camera path JSON.</h3>'))

%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename

%cd /content/
!ns-render --load-config $config_filename --traj filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4

# Use Google Drive to save our rendered bg video
%cd /content
!mkdir -p 'gdrive/My Drive/nerfstudio/renders'
!cp renders/output.mp4 'gdrive/My Drive/nerfstudio/renders/output.mp4'

🎥 Rendering 🎥 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0.15 fps 27:16


In [1]:
from google.colab import files

In [3]:
#@title # Composite Blender animation with NeRF Background Video { vertical-output: true }
#@markdown <h3>Render the Blender animation, then run this cell and upload it here.</h3>
#@markdown <h5>The final video should be at renders/nerfstudio+blender.mp4!</h5>

from IPython.core.display import display, HTML
display(HTML('<h3>Upload the blender animation with a clear background.</h3>'))

%cd /content/
uploaded = files.upload()
blender_video_filename = list(uploaded.keys())[0]

%cd /content/
!ffmpeg -i renders/output.mp4 -i $blender_animation.mp4 -filter_complex "overlay" "renders/nerfstudio+blender.mp4"

# Use Google Drive to save our final video
%cd /content
!mkdir -p '/content/gdrive/My Drive/nerfstudio/renders'
!cp "renders/nerfstudio+blender.mp4" "gdrive/My Drive/nerfstudio/renders/nerfstudio+blender.mp4"

/content


KeyboardInterrupt: ignored